In [40]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.model_selection import train_test_split

#load the dataset
Train_data=pd.read_csv('train.csv')
Test_data=pd.read_csv('test.csv')


#normalize the data set
Train_data_normalized = Train_data.divide(Train_data.max())
Test_data_normalized = Test_data.divide(Test_data.max())



#split the dataset
X = Train_data_normalized.iloc[:, :-1].values
Y = Train_data_normalized.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)


In [41]:
#Two types of Nodes (decision,leaf)
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        # for decision node
        self.feature_index = feature_index  
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [42]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):                         #constractour
        self.root = None    
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
        

    def build_tree(self, dataset, curr_depth=0):                       # recursive function to build the tree 
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)

        # split until violate stopping conditions
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:

            best_split = self.get_best_split(dataset, num_samples, num_features)            # find the best split
            # check if information gain is positive 
            if best_split["info_gain"]>0:  #best split is dectionary                                                
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    
    
    

    def get_best_split(self, dataset, num_samples, num_features):    #function to find the best split
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    
    
    
    
    def split(self, dataset, feature_index, threshold):   #function to split the data 
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    
    
    
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):    #function to compute information gain
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    
    
    
    def entropy(self, y):                        #function to compute entropy
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    
    
    
    
    def gini_index(self, y):                        #function to compute gini   
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
        
        
        
    def calculate_leaf_value(self, Y):        #function to compute leaf node Label
        Y = list(Y)
        return max(Y, key=Y.count)
    
    
    
    
    def print_tree(self, tree=None, indent=" "):      #function to print the tree for make me Confident.     
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)
        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    
    
    
    
    def fit(self, X, Y):         #function to train the tree by calling build_tree Function
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    
    
    
    
    
    def predict(self, X):           #function to predict test dataset 
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):        #function to predict a single feature
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [43]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=7)
classifier.fit(X_train,Y_train)

In [44]:
classifier.print_tree()

X_13 <= 0.5757878939469735 ? 0.18293403724472734
 left:X_13 <= 0.27388694347173587 ? 0.22188692880148597
  left:X_11 <= 0.7377551020408163 ? 0.02433867531170028
    left:X_0 <= 0.9074074074074074 ? 0.022426195630410414
        left:X_13 <= 0.26313156578289143 ? 0.0075483822886870805
                left:X_6 <= 0.984375 ? 0.006604911232198998
                                left:X_0 <= 0.8303303303303303 ? 0.0008924646115540599
                                                                left:X_0 <= 0.7497497497497497 ? 0.00017507452713818109
                                                                                                                                left:0.0
                                                                                                                                right:0.0
                                                                right:X_11 <= 0.6178571428571429 ? 0.06816568047337264
                                                       

In [45]:
Y_pred = classifier.predict(X_test) 



In [46]:
Y_test = Y_test.flatten()
Y_test = Y_test.tolist()


In [47]:
Y_pred
len(Y_pred)

400

In [48]:
Y_test
len(Y_test)

400

In [49]:
def calculate_accuracy(predictions, labels):
    correct = 0
    total = len(labels)

    for i in range(total):
        if predictions[i] == labels[i]:
            correct += 1

    return (correct / total) * 100

In [50]:
Accuracy=calculate_accuracy(Y_test,Y_pred)
print("The Accuracy is "+str(calculate_accuracy(Y_test,Y_pred)) + '%')

The Accuracy is 81.5%
